# Dense Grid for US Lower 48

**Analyst:** Taryn Fransen

**Shapefile source:** US Census Bureau

**Shapefile download URL:** https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html

**Shapefile location:** /shares/maps100/data/raw/cb_2018_us_nation_5m/cb_2018_us_nation_5m.shp

**Output:** 
* csv of 8201124 points at .01-degree resolution over the landmass of the US lower 48 states
* Location: /shares/maps100/data/output/grid/us_dense_grid.csv

**Scope:**
* north = 49.3457868 # north lat
* west = -124.7844079 # west long
* east = -66.9513812 # east long
* south =  24.7433195 # south lat

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely
from shapely import geometry
from shapely.geometry import Point, MultiPoint, Polygon, MultiPolygon, asPoint
from shapely.ops import cascaded_union
from shapely.ops import prep
from shapely.vectorized import contains
from shapely.prepared import prep
import matplotlib.pyplot as plt
import math as math
import pickle

In [ ]:
shp_path = '/shares/maps100/data/raw/cb_2018_us_nation_5m/cb_2018_us_nation_5m.shp'

In [ ]:
state_map = gpd.read_file(shp_path).explode().reset_index()

In [ ]:
north = 49.3457868 # north lat
west = -124.7844079 # west long
east = -66.9513812 # east long
south =  24.7433195 # south lat
state_map = state_map.cx[west:east, south:north].reset_index()

In [ ]:
delta = .01
half_delta = delta/2

In [ ]:
def make_grid(latmin, latmax, lonmin, lonmax, res = delta):
    latVals = []
    currentLat = latmax
    i = 0
    while(currentLat > latmin + res):
      latVals.append(currentLat)
      i = i + 1
      currentLat = currentLat - res
    latVals = np.array(latVals)
    
    lonVals = []
    currentLon = lonmin
    i = 1
    while(currentLon < (lonmax - res)):
      lonVals.append(currentLon)
      i = i + 1
      currentLon = currentLon + res
    lonVals = np.array(lonVals)
    
    #shift to make the values represent grid cell centers
    latVals = latVals - res/2.0
    lonVals = lonVals + res/2
    
    grid = {"lat" : latVals,
           "lon" : lonVals}
    
    #Return the grid in degrees: 
    return grid

In [ ]:
#grid = make_grid(24.745, 49.355, -124.795, -66.955)
grid = make_grid(24.74, 49.36, -124.80, -66.95)

In [ ]:
def landmassOnlyGrid(lats, lons, gpdFile):
    """
    Takes a grid in the form of two arrays, lats and lons. First, transforms the arrays into a flat grid. 
    Then checks for point intersections in the attached geopandas file.
    
    Returns a flat grid dictionary object with only lats and lons that are included in the gpd geometry file.
    .
    Needs shapely, geopandas, and numpy.
    """
    gpdFile["prep"] = gpdFile["geometry"].apply(prep) # prepare the geometry to improve speed
    print('prepped')
    
    grid_lats, grid_lons = np.meshgrid(lats, lons) # Create grid from input arrays
    flat_lats = grid_lats.flatten() #Making two arrays that together correspond to all of the grid points
    flat_lons = grid_lons.flatten()
    
    points = [Point((flat_lons[i], flat_lats[i])) for i in range(len(flat_lats))] # turn each point into a Shapely object
    print('got points')
    
    #total = str(len(gpdFile))
    for i in range(len(gpdFile)):
        #log_text("Loop status: " + str(i) + " out of " + total)
        print(i)
        prepared_polygon = gpdFile["prep"][i]

        intersect_points = list(filter(prepared_polygon.contains, points))

        if i == 0:
            hits = intersect_pointsx
        else:
            hits = hits + intersect_points

    output_lons = []
    output_lats = []

    for i in range(len(hits)):
        output_lons.append(hits[i].x)
        output_lats.append(hits[i].y)

    landGridFlat = {    #Note that this output will be the full length 'flat' grid as json file. 
        "lat" : output_lats,
        "lon" : output_lons,
        }
    
    return pd.DataFrame(landGridFlat) #currently the output is not ordered. This improved runtime.

In [ ]:
land_grid = landmassOnlyGrid(grid['lat'], grid['lon'], state_map) # takes 1-2 hours

land_grid_df = land_grid.sort_values(["lat","lon"])
land_grid_df = land_grid_df.reset_index(drop=True)
land_grid_df = land_grid_df.round(3)

output_path = '/shares/maps100/data/output/grid/'
file_name = output_path + 'us_dense_grid_1' + ".csv"

land_grid_df.to_csv(file_name, sep=',',index=False)

In [ ]:
# read the grid back in from the csv

output_path = '/shares/maps100/data/output/grid/'
file_name = output_path + 'us_dense_grid_1' + ".csv"
us_dense_grid = pd.read_csv(file_name)

In [ ]:
# plot the grid to make sure it looks right

x = us_dense_grid['lon']
y = us_dense_grid['lat']

plt.figure(figsize=(20,10))
plt.scatter(x,y, s = .00001)

for i in range(0,len(state_map)):
    y,x=state_map['geometry'][i].exterior.coords.xy
    plt.plot(y,x,color='grey')
    

In [ ]:
def get_box(lon, lat, delta):
    half_delta = delta/2
    upper_left = geometry.Point(lon-half_delta,lat+half_delta)
    upper_right = geometry.Point(lon+half_delta, lat+half_delta)
    lower_left = geometry.Point(lon-half_delta, lat-half_delta)
    lower_right = geometry.Point(lon+half_delta, lat-half_delta)
    pointList = [upper_left,upper_right,lower_right,lower_left]
    poly = geometry.Polygon(pointList)
    return poly

In [ ]:
poly_grid = us_dense_grid.copy()
poly_grid['box'] = poly_grid.apply(lambda row: get_box(row['lon'],row['lat'],delta), axis=1)
poly_grid = gpd.GeoDataFrame(us_dense_grid).set_geometry('box')

In [ ]:
with open('/shares/maps100/data/output/grid/us_dense_grid_polys_1', 'wb') as f:
    pickle.dump(poly_grid, f)